### IMPORT DE BIBLIOTECAS

In [5]:
from collections import defaultdict
import pandas as pd
import numpy as np
import os

import implicit
from implicit.evaluation import (train_test_split, 
                                 ndcg_at_k, AUC_at_k,
                                 mean_average_precision_at_k, 
                                 precision_at_k)

from implicit.nearest_neighbours import bm25_weight

from scipy.sparse import (csr_matrix, 
                          save_npz, 
                          load_npz)
import scipy.stats

from matplotlib import pyplot as plt
import seaborn as sns

# from surprise import Dataset, Reader, SVD, SVDpp, KNNWithMeans
# from surprise.model_selection import train_test_split, cross_validate
# from surprise import accuracy
# from surprise.model_selection import cross_validate

# from sklearn.neighbors import NearestNeighbors
# from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
import pickle
from tqdm import tqdm
import warnings

In [6]:
warnings.filterwarnings('ignore')

### Carregamento de dados de categorias e histórico de clicks

In [7]:
df_de_cat = pd.read_csv(os.getcwd()+'\.txt\category_de.txt') # DataFrame Informativo das categorias

In [8]:
clicks_de = pd.read_csv(os.getcwd()+'\.txt\clicks_de_sample_2.txt', sep = ',', header=0) # df de histórico de clicks
clicks_de.columns

Index(['UserId', 'OfferId', 'OfferViewId', 'CountryCode', 'Category', 'Source',
       'UtcDate', 'Keywords', 'OfferTitle'],
      dtype='object')

Valores missing na coluna de OfferTitle que serão excluídos.

In [9]:
clicks_de = clicks_de[clicks_de.OfferTitle.isna() == False]

### Conversão das colunas de usuário e oferta em categórica e criando novas colunas com os códigos adotados 

In [10]:
clicks_de.UserId = clicks_de.UserId.astype('category')
clicks_de.OfferId = clicks_de.OfferId.astype('category')

In [11]:
clicks_de['User'] = clicks_de.UserId.cat.codes
clicks_de['Offer'] = clicks_de.OfferId.cat.codes

### CRIAÇÃO DE COLUNA DE CLICKS POR CATEGORIA

In [12]:
#clicks_de.drop(columns = ['Keywords'], axis = 1, inplace=True)  # DROP DAS KEYWORDS

#df_de_cat.rename({'Ancertor_ID':'Ancestor_ID'}, axis = 1, inplace = True) # RENAME DO ANCESTOR_ID

#df_de_cat.drop('Unnamed: 0', axis =1, inplace = True) # Remoção de coluna Unnamed

clicks_de['Cat_clicks'] = clicks_de.groupby('Category')['OfferId'].transform('count') # Criação de coluna de clicks por categoria

### Merge do Dataframe de categorias com o dataframe de clicks 

In [14]:
clicks_de = clicks_de.merge(df_de_cat, left_on = 'Category', right_on = 'ID').drop(['ID'], axis = 1) 

### Criação de coluna com o nº total de clicks do usuário e nº total de clicks do produto

In [15]:
clicks_de['UserTotalClicks'] = clicks_de.groupby(by=['UserId'])['OfferId'].transform('count')

In [16]:
clicks_de['ProductClicks'] = clicks_de.groupby(by='OfferId')['UserId'].transform('count')

In [21]:
#clicks_de_filtered.UserId = clicks_de_filtered.UserId.astype('category')
# clicks_de_filtered.OfferId = clicks_de_filtered.OfferId.astype('category')


# clicks_de_filtered['User'] = clicks_de_filtered.UserId.cat.codes
# clicks_de_filtered['Offer'] = clicks_de_filtered.OfferId.cat.codes#Cap minimo de clicks para integrar o sistema de recomendação

In [22]:
clicks_de_filtered = clicks_de[(clicks_de.UserTotalClicks > 10) & (clicks_de.ProductClicks > 50) & (clicks_de.ProductClicks < (clicks_de.ProductClicks.mean() + clicks_de.ProductClicks.std()))]

In [23]:
# clicks_de_filtered.UserId = clicks_de_filtered.UserId.astype('category')
# clicks_de_filtered.OfferId = clicks_de_filtered.OfferId.astype('category')


# clicks_de_filtered['User'] = clicks_de_filtered.UserId.cat.codes
# clicks_de_filtered['Offer'] = clicks_de_filtered.OfferId.cat.codes


### Agrupamento de dos clicks de usuário em ofertas únicas para termos a quantidade de cada usuário em cada oferta.


In [24]:
clicks_per_user_product = clicks_de_filtered.groupby(by=['User','Offer']).count()['UserTotalClicks'].reset_index().rename({'UserTotalClicks':'UserClicks'}, axis = 1)

In [25]:
#clicks_per_user_product.UserClicks = 1

### Criação de matrizes esparsas Usuário-item e item-usuário

In [26]:
alpha = 40
sparse_item_user = csr_matrix((clicks_per_user_product['UserClicks'], (clicks_per_user_product['Offer'], clicks_per_user_product['User'])))
sparse_user_item = csr_matrix((clicks_per_user_product['UserClicks'], (clicks_per_user_product['User'], clicks_per_user_product['Offer'])))

sparse_user_item = (sparse_user_item).astype('double') # Conversão de tipo para que o modelo ALS funcione corretamente
data = (sparse_item_user).astype('double') # Conversão de tipo para que o modelo ALS funcione corretamente

In [27]:
# weight the matrix, both to reduce impact of users that have played the same artist thousands of times
# and to reduce the weight given to popular items
item_user_clicks = bm25_weight(data, K1=100, B=0.8)
# get the transpose since the most of the functions in implicit expect (user, item) sparse matrices instead of (item, user)
user_clicks = item_user_clicks.T.tocsr()

In [28]:
#Esparsidade de matriz
possible_interactions = sparse_item_user.shape[0]*sparse_item_user.shape[1]
interacted = len(sparse_item_user.nonzero()[0])
sparsity = 1 - interacted/possible_interactions
sparsity

0.9999999722688098

 Mais de 99.99% das interações possíveis entre usuários e produtos na atual base dados não foi ainda realizada. Segundo artigo: For collaborative filtering to work, the maximum sparsity you could get away with would probably be about 99.5% or so. Devemos reavaliar a matriz?

In [29]:
save_npz(os.getcwd()+"/.npz/sparse_user_item.npz", sparse_user_item)
save_npz(os.getcwd()+"/.npz/sparse_item_user.npz", sparse_item_user)

In [30]:
model_path = os.getcwd()+'/.pkl/de_als_model.pkl'

* Criação de diferentes matrizes esparsas para operar com o algoritmo. Usuário-item e item-usuário. Cada uma deve ser usada no momento preciso
* O alfa é o coeficiente de confiabilidade da interação do usuário com um item específico. Valor utilizado fi adotado com base no artigo: https://towardsdatascience.com/alternating-least-square-for-implicit-dataset-with-code-8e7999277f4b. Mas, podemos testar outros valores na validação do modelo.
* Outro artigo de base pra elaboração do modelo: https://medium.com/analytics-vidhya/implementation-of-a-movies-recommender-from-implicit-feedback-6a810de173ac

# FUNÇÃO DE RECOMENDAÇÕES - IMPLICIT

## Treinamento de modelos

In [31]:
offers = pickle.load(open(os.getcwd()+"/.pkl/offers.pkl", "rb"))

Carregamentodo dicionário que converte os códigos de ofertas para o seu título de oferta. Ainda falta traduzir do alemão para o inglês para tirar mais significado dos resultados

In [48]:
def als_model():
    
    '''computes p@k and map@k evaluation metrics and saves model'''
    
    #sparse_item_user = load_npz(os.getcwd()+"/.npz/sparse_item_user.npz")
      
    train, test = implicit.evaluation.train_test_split(sparse_user_item, train_percentage=0.9)

    model = implicit.als.AlternatingLeastSquares(factors=10, 
                                                 regularization=1, 
                                                 iterations=50,
                                                 calculate_training_loss=False)
    alpha=15
    model.fit(train*alpha)

    with open(model_path, 'wb') as pickle_out:
        pickle.dump(model, pickle_out)
    
    return train, test, model

In [49]:
train, test, model = als_model()
implicit.evaluation.precision_at_k(model,train,test)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/340 [00:00<?, ?it/s]

0.4153846153846154

# Avaliação Baseline

In [128]:
def get_k_most_popular(sparse_item_user, k):
    
    """Retorna um array com os códigos dos itens clicados por mais clientes da base de dados"""
    
    new_sparse = sparse_item_user.copy() # Nova esparsa para não modificar a original
    interaction = np.ones_like(new_sparse.data) # Altera os valores da matriz esparsa para '1' se houve interação para que não sejam contadas múltiplas interações do mesmo cliente com o mesmo produto
    new_sparse.data = interaction # Assimila o array de interações de 1's aos dados da matriz esparsa
    pop_items = np.array((new_sparse).sum(axis = 1)).reshape(-1) # Aqui é feita a soma de interações que ocorreram em cada uma das linhas, que correspondem a cada um dos produtos
    most_popular = pd.Series(pop_items).sort_values(ascending = False)[:k] # Os itens são convertidos em série para que possam ser ordenados e então captados os deus com maior incidência para que sejam retornados seus indices(código do produto)
    
    return most_popular.index

 O modelo de baseline considerará como sugestão os produtos mais populares de todo o dataframe como recomendação para todos os clientes

In [263]:
def get_top_k(userid, sparse_user_item, k=10):
    
    '''Retorna os top-K produtos clicados por um usuário de acordo 
    com o nível de confiança obtido através das suas interações'''
    top_K =pd.Series(sparse_user_item[userid].data, index = sparse_user_item[userid].indices).sort_values(ascending = False)[:k]
    return top_K.index

In [264]:
get_top_k(8, sparse_user_item)

Int64Index([200549], dtype='int64')

In [50]:
def baseline_precision_at_k(sparse_item_user, k=10, test_pct=0.2):

    """Avalia a precisão caso o modelo recomendasse apenas os itens mais populares a todos os clientes.
    A métrica é calculada com base em uma seleção aleatória de 20% dos clientes."""
    
    # Seleção aleatória de 20% dos usuários para testagem por popularidade
    test_sample = np.random.choice(sparse_item_user.indices,
                                   size = int(test_pct*len(sparse_item_user.indices)), 
                                   replace = False) 
    
    most_popular = (get_k_most_popular(sparse_item_user, k)) # lista de 10 itens mais populares
    
    total_precision = 0 
    sparse_user_item = sparse_item_user.T.tocsr()
    
    for user in test_sample:
        relevance = 0
        top_K = get_top_k(user, sparse_user_item, k)
        
        for item in most_popular:
            if item in top_K: # Alteração do pesoa das interações para 0 e 1. Assim, será contabilizado apenas se o usuário interagiu com o item ou não.
                interact = 1
            else:
                interact = 0
            relevance+= interact
            
        relevance/=k
        total_precision+=relevance
        
    total_precision/=len(test_sample)
    
    return total_precision

In [51]:
baseline_precision_at_k(sparse_user_item.T.tocsr())

NameError: name 'get_k_most_popular' is not defined

### Avaliação do modelo

In [131]:
baseline_precision_at_k(sparse_item_user, k=10)

NameError: name 'get_top_k' is not defined

In [52]:
def model_evaluation(train, test, model, k=10): 
    
    '''Avaliação do modelo treinado com as funções da biblioteca Implicit.
    Retorna dicionário com p@k, map@k, ndcg@k e auc@k.'''

    
    p_at_k = implicit.evaluation.precision_at_k(model, train_user_items=train, 
                                                test_user_items=test,
                                                K=k, 
                                                show_progress = False)
    
    m_at_k = implicit.evaluation.mean_average_precision_at_k(model, 
                                                             train_user_items = train, 
                                                             test_user_items = test, 
                                                             K=k, 
                                                             show_progress = False)

    ndcg_at_k = implicit.evaluation.ndcg_at_k(model, 
                                              train_user_items = train,
                                              test_user_items = test, 
                                              K=k, 
                                              show_progress = False)

    auc_at_k = implicit.evaluation.AUC_at_k(model, 
                                            train_user_items = train, 
                                            test_user_items = test, 
                                            K=k, 
                                            show_progress = False)
    metrics = {'p@K':p_at_k, 
               'map@K': m_at_k, 
               'ndcg@K':ndcg_at_k, 
               'auc@K':auc_at_k}
    
    return metrics

In [53]:
metrics = model_evaluation(train, test, model)

In [54]:
metrics

{'p@K': 0.4153846153846154,
 'map@K': 0.1996994631185809,
 'ndcg@K': 0.24583050569216344,
 'auc@K': 0.6921442264252101}

* Sobre metricas de precisão @k: https://medium.com/@m_n_malaeb/recall-and-precision-at-k-for-recommender-systems-618483226c54
* Sobre NDCG: https://towardsdatascience.com/evaluate-your-recommendation-engine-using-ndcg-759a851452d1
* Sobre Mean Average Precision: https://towardsdatascience.com/breaking-down-mean-average-precision-map-ae462f623a52

Ainda não foi realizada qualquer tunagem de hiperaparâmetros. Podemos pegar alguns valores de referencia para rodar um gridsearch

## Funções de recomendações

In [193]:
def recommend(user):
    
    ''' Retorna uma lista de itens recomendados para o usuário dado de acordo com a biblioteca Implicit.
        Também é retornado uma lista com os itens já clicados por esse usuário'''
    
    #sparse_user_item = load_npz("/.npz/sparse_user_item.npz")
    
    with open(model_path, 'rb') as pickle_in:
        model = pickle.load(pickle_in)
    
    if user not in sparse_user_item.T.tocsr().indices:
        return "Invalid User"
        
    recommended, _ = (model.recommend(user, sparse_user_item[user]))

    original_user_items = list(sparse_user_item[user].indices)

    return recommended, original_user_items

In [196]:
sparse_user_item.T.tocsr().indices

array([ 39584,  42561,  58749, ..., 363287, 369503, 408904], dtype=int32)

In [194]:
def most_similar_items(item_id, n_similar=10):
    '''computes the most similar items'''
    
    with open(model_path, 'rb') as pickle_in:
        model = pickle.load(pickle_in)

    similar, score = zip(*model.similar_items(item_id, n_similar)[1:])

    return similar

In [271]:
a = recommend(299228)

In [272]:
[offer_cat[item] for item in a[0]]

[100360623,
 100332323,
 100332323,
 100332323,
 125801,
 100332323,
 100332323,
 146501,
 100332323,
 100332323]

In [273]:
[offer_cat[item] for item in a[1]]

[100334223, 117001, 100332323]

In [241]:
sparse_item_user.indices[20:30]

array([118613, 150977, 162387, 167336, 245902, 299228, 300016, 312634,
       319826, 340386], dtype=int32)

In [43]:
def most_similar_users(user_id, n_similar=10):
    '''computes the most similar users'''
    sparse_user_item = load_npz(os.getcwd()+"/.npz/sparse_user_item.npz")
    
    with open(model_path, 'rb') as pickle_in:
        model = pickle.load(pickle_in)

    similar, _ = zip(*model.similar_users(user_id, n_similar)[1:])

    # original users items
    original_user_items = list(sparse_user_item[user_id].indices)
    
    common_items_users = {}

    # now we want to add the items that a similar user has rated
    for user in similar:
        # Verifica em cada usuário considerado similar quais são os itens que estes
        # tem em comum com o usuário selecionado
        common_items_users[user] = set(list(sparse_user_item[user].indices)) & set(original_user_items)
    
    # retorna usuários similares, e quais são os itens comuns correspondentes a cada um desses usuários
    return similar, common_items_users

In [ ]:
def recalculate_user(user_ratings):
    '''adds new user and its liked items to sparse matrix and returns recalculated recommendations
       Receives the user clicked products vector (user_ratings)''' 

    m = load_npz('sparse_user_item.npz')
    n_users, n_movies = m.shape

    ratings = [alpha for i in range(len(user_ratings))]

    m.data = np.hstack((m.data, ratings))
    m.indices = np.hstack((m.indices, user_ratings))
    m.indptr = np.hstack((m.indptr, len(m.data)))
    m._shape = (n_users+1, n_movies)

    # recommend N items to new user
    with open(model_path, 'rb') as pickle_in:
        model = pickle.load(pickle_in)
        
    recommended, _ =  zip(*model.recommend(n_users, m, recalculate_user=True))
    
    return recommended

* A matriz m passa a ser a matriz com o novo usuário atualizado e é levada em consideração no para o cálculo de novos vetores.

Nota: 
* Após os ajustes na organização das matrizes esparsas, o modelo parece não mais repetir recomendações de itens que já foram clicados pelo usuário
* O modelo parece também não mais necessitar de tradução dos códigos de ofertas e usuário adotados na matriz esparsa para os códigos da matriz original

### Criação e armazenamento do dicionário código-titulo de oferta (carregado na parte de cima do código).

In [156]:
df_temp = clicks_de[['Offer','OfferTitle']]

In [161]:
df_temp = df_temp.drop_duplicates(['Offer','OfferTitle'])

In [169]:
teste_dicio = dict(zip(df_temp['Offer'], df_temp['OfferTitle']))

In [95]:
dic = pd.read_csv('de_traduzido.csv')
dic['Offer2'] = dic['OfferTitle'].map(dict(zip(clicks_de.OfferTitle, clicks_de.Offer)))

In [115]:
pickle.dump(offers, open('.pkl/offers.pkl', 'wb'))

### Criação e armazenamento do dicionário código_oferta -> Categoria_1 (carregado na parte de cima do código).

In [268]:
df_temp = clicks_de[['Offer','Category']]

In [269]:
df_temp = df_temp.drop_duplicates(['Offer','Category'])

In [270]:
offer_cat = dict(zip(df_temp['Offer'], df_temp['Category']))

### Tunagem de hiperparametros

In [214]:
grid = {'factors': [10,50,100,200],
       'regularization':[1,0.5, 0.1, 0.01],
        'iterations':[10,30,50, 100], 
        'alphas':[1,15,40,60, 100]}

In [215]:
model = implicit.als.AlternatingLeastSquares()

In [246]:
def gridsearch_als(grid):
    results = []
    for factor in grid['factors']:
        for regularization in grid['regularization']:
            for iteration in grid['iterations']:
                for alpha in grid['alphas']:
            
                    model = implicit.als.AlternatingLeastSquares(factors = factor, 
                                                                 regularization = regularization, 
                                                                 iterations = iteration)
                    model.fit(train*alpha)

                    partial = model_evaluation(train, test, model, )
                    results.append([factor, regularization, iteration, alpha, partial['p@K'],partial['map@K'], partial['ndcg@K'], partial['auc@K']])
                
    final = pd.DataFrame(results, columns = ['Factors','Regularization','Iteration','alpha','P@K','MAP@K','NDCG@K','AUC@K'])
    return final
                

In [ ]:
grid_results = gridsearch_als(grid)

In [248]:
grid_results.to_csv('params.csv')

In [249]:
grid_results.sort_values(by=['P@K','NDCG@K'], ascending = False)

,Factors,Regularization,Iteration,alpha,P@K,MAP@K,NDCG@K,AUC@K
10,10,1.00,50,1,0.277019,0.118268,0.161464,0.625906
75,10,0.01,100,1,0.277019,0.112387,0.157666,0.626029
30,10,0.50,50,1,0.276451,0.115258,0.158532,0.624036
50,10,0.10,50,1,0.275882,0.117437,0.161001,0.626247
55,10,0.10,100,1,0.270762,0.113639,0.156124,0.623370
...,...,...,...,...,...,...,...,...
4,10,1.00,10,100,0.088168,0.017550,0.035890,0.538423
304,200,0.01,10,100,0.072241,0.019690,0.034520,0.530543
64,10,0.01,10,100,0.072241,0.010721,0.025446,0.530581
144,50,0.01,10,100,0.070535,0.020357,0.033903,0.528062
